In [1]:
import re
import time

import numpy as np
import pandas as pd
import requests

In [4]:
!pip install progressbar2
from progressbar import progressbar

## Scraping

### Scraping Functions

In [5]:
# function to scrape reddit page (takes a reddit .json url)
# returns posts 

def scraper_bike(url):
    headers = {'User-Agent' : 'override this bad boy!'}
    posts = []
    after = {}

    for page in progressbar(range(40)):
        params = {'after': after}
        pagepull = requests.get(url=url, params=params, headers=headers)
        page_dict = pagepull.json()
        posts.extend(page_dict['data']['children'])
        after = page_dict['data']['after']
        time.sleep(.2)
        
    return posts

In [6]:
# function to convert posts to DataFrame - won't allow duplicate posts since unique id 'name' is set as index
# Extract: name (as index) and subreddit, selftext, title (as columns)

def posts_to_df(post_list):
    i = 0
    post_dict = {}
    
    for post in post_list:
        ind = post_list[i]['data']
        post_dict[ind['name']] = [ind['subreddit'], ind['title'], ind['selftext']]
        i += 1

    df_name = pd.DataFrame(post_dict)
    df_name = df_name.T
    df_name.columns = ['subreddit', 'title', 'selftext']
    
    return df_name

In [7]:
# takes scraper function and url - outputs dataframe

def scrape_to_df(scrape_func, url):
    
    return posts_to_df(scrape_func(url))

In [8]:
#### If you want to scrape repeatedly over time and add to a csv
# scrape, import csv, concat, drop duplicate, and output to csv
# takes in scraper function, url, csv filename to import, csv filename to output
# Outputs - Concatenated DataFrame as csv

def scrape_add(scrape_func, url, import_file, export_file):
    scrape_df = posts_to_df(scrape_func(url))
    imported_df = pd.read_csv(import_file, index_col = 'Unnamed: 0')
    concat_df = pd.concat([imported_df, scrape_df])
    concat_df = concat_df[~concat_df.index.duplicated(keep='first')]
    concat_df.to_csv(export_file)

### Run Scrape

In [9]:
# You can also put in any 2 subreddits in as the URL and get results for those

nfltest = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/nfl.json')
nbatest = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/nba.json')

100% (40 of 40) |########################| Elapsed Time: 0:00:24 Time:  0:00:24
100% (40 of 40) |########################| Elapsed Time: 0:00:22 Time:  0:00:22


In [10]:
politics_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/politics.json')
conservative_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/conservative.json')

100% (40 of 40) |########################| Elapsed Time: 0:00:23 Time:  0:00:23
100% (40 of 40) |########################| Elapsed Time: 0:00:24 Time:  0:00:24


In [11]:
nbatest.shape

(786, 3)

In [12]:
nfltest.shape

(942, 3)

##### These scrape_add functions add to already built csvs

In [13]:
# scrape_add(scraper_bike, 'https://www.reddit.com/r/CollegeBasketball/new.json', 'NCAA_Posts_Update2.csv', 'NCAA_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/AskScience/new.json', 'AskSci_Posts_Update2.csv', 'AskSci_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/nba/new.json', 'NBA_Posts_Update2.csv', 'NBA_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/nfl/new.json', 'NFL_Posts_Update2.csv', 'NFL_Posts_Update3.csv')

### Data Cleaning / Preprocessing

In [14]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

pd.set_option('max_colwidth', 300)

In [15]:
# drop column

nfltest = nfltest.drop(columns = 'selftext')
nbatest = nbatest.drop(columns = 'selftext')

In [16]:
# merge subreddit data

train = pd.concat([nfltest, nbatest])

In [17]:
train

,subreddit,title
t3_ei5he6,nfl,Talko Tuesday
t3_ehwkan,nfl,/r/NFL Best Of 2019 Nomination Thread
t3_ei32ez,nfl,"[Silver] The Ron Rivera deal is done. He has reached an agreement to become the new coach of the Redskins, according to a source familiar with negotiations. @AroundTheNFL @nflnetwork"
t3_ei2x87,nfl,"[Schefter] Giants are expected to interview former Packers’ head coach Mike McCarthy this weekend, per source."
t3_ei7htz,nfl,"[Schefter]John Dorsey is out as the Browns’ GM, per source. Two sides couldn’t come to an agreement on a future re-structure of the organization."
t3_ei3j1c,nfl,"[Schefter] Baylor HC Matt Rhule declined the chance to interview for the Cleveland Browns head coaching job, per league sources. Rhule is expected to be a leading candidate for the Giants’ and Panthers’ HC jobs."
t3_ei3x5k,nfl,"[Schefter] Jaguars’ HC Doug Marrone is returning for the 2020 season, per sources."
t3_ei2us9,nfl,"On the one hand, we're due a Super Bowl run from a team outside of the top two seeds. The last time was 2012."
t3_ei5tnj,nfl,[Serious] CJ Anderson says on Twitter that his brother has been killed
t3_ei4c4e,nfl,[Campbell] Bears GM Ryan Pace definitively commits to Mitch Trubisky as Bears QB1 in 2020.


##### Tokenize (grab only word characters)

In [18]:
word_tokenizer = RegexpTokenizer(r'\w+')

In [19]:
train['title'] = train['title'].map(lambda x: word_tokenizer.tokenize(x.lower()))

In [20]:
# rejoin list of tokenized words into single string for each row

train['title'] = train['title'].map(lambda x: ' '.join(x))

In [21]:
train['title'][0:5]

t3_ei5he6                                                                                                                                                                      talko tuesday
t3_ehwkan                                                                                                                                               r nfl best of 2019 nomination thread
t3_ei32ez    silver the ron rivera deal is done he has reached an agreement to become the new coach of the redskins according to a source familiar with negotiations aroundthenfl nflnetwork
t3_ei2x87                                                                          schefter giants are expected to interview former packers head coach mike mccarthy this weekend per source
t3_ei7htz                                       schefter john dorsey is out as the browns gm per source two sides couldn t come to an agreement on a future re structure of the organization
Name: title, dtype: object

In [22]:
train['title'][0:5]

t3_ei5he6                                                                                                                                                                      talko tuesday
t3_ehwkan                                                                                                                                               r nfl best of 2019 nomination thread
t3_ei32ez    silver the ron rivera deal is done he has reached an agreement to become the new coach of the redskins according to a source familiar with negotiations aroundthenfl nflnetwork
t3_ei2x87                                                                          schefter giants are expected to interview former packers head coach mike mccarthy this weekend per source
t3_ei7htz                                       schefter john dorsey is out as the browns gm per source two sides couldn t come to an agreement on a future re structure of the organization
Name: title, dtype: object

### Train test split and converting series to list of strings then to array

In [23]:
X = train[['title']]
y = train['subreddit']

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.25,
                                                    random_state=42,
                                                    stratify=y)

In [25]:
# baseline is

y.value_counts(normalize=True)

nfl    0.545139
nba    0.454861
Name: subreddit, dtype: float64

In [26]:
# create our training data list - this is a list of strings, with each string being a post title

clean_train_data = []

for traindata in X_train['title']:
    clean_train_data.append(traindata)

In [27]:
len(clean_train_data)

1296

In [28]:
# create test data list

clean_test_data = []

for testdata in X_test['title']:
    clean_test_data.append(testdata)

In [29]:
len(clean_test_data)

432

### Count Vectorizer

In [30]:
# instantiate our CountVectorizer. This counts the number of appearances of all the words in our training data and
# eliminates common english stop words. 5000 max features works well for our purposes (tested various numbers). Our
# data is already preprocessed and tokenized manually earlier. ngram_range is 1,3, although all or nearly all our
# features are single words

vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words='english',
                             max_features=5000,
                             ngram_range=(1, 3))

In [31]:
# fit our training data and test data lists to our count_vectorizer

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

In [32]:
# convert to array

train_data_features = train_data_features.toarray()

In [33]:
# check shapes

train_data_features.shape, test_data_features.shape

((1296, 5000), (432, 5000))

In [34]:
# I wanted check that the features corpus was as expected - removed print statement for readability

vocab = vectorizer.get_feature_names()

In [35]:
vocab[0:200]

['000',
 '000 bonus',
 '000 yards',
 '10',
 '10 22',
 '10 22 defeat',
 '10 23',
 '10 consequential',
 '10 consequential athletes',
 '10 december',
 '10 december 29',
 '10 defeat',
 '10 los',
 '10 los angeles',
 '10 rebs',
 '100',
 '1000',
 '1000 yard',
 '100th',
 '102',
 '104',
 '10th',
 '11',
 '11 13',
 '11 20',
 '11 98',
 '11 98 97',
 '11 field',
 '11 miami',
 '11 miami heat',
 '11 point',
 '110',
 '112',
 '113',
 '113 point',
 '113 point differential',
 '115',
 '116',
 '117',
 '118',
 '119',
 '12',
 '12 20',
 '12 21',
 '12 22',
 '12 27',
 '12 27 2019',
 '12 28',
 '12 28 vs',
 '12 29',
 '12 37',
 '12 sacks',
 '12 sacks adrian',
 '120',
 '122',
 '123',
 '129',
 '12th',
 '12th head',
 '12th head coach',
 '13',
 '13 20',
 '13 21',
 '13 games',
 '13 record',
 '13 team',
 '14',
 '14 18',
 '14 18 december',
 '14 19',
 '141',
 '14th',
 '15',
 '15 defeat',
 '15th',
 '16',
 '16 15',
 '16 game',
 '16 game season',
 '17',
 '17 defeat',
 '17 seasons',
 '18',
 '18 amp',
 '18 december',
 '18 point

## MODELING

### Logistic Regression

In [36]:
from sklearn.linear_model import LogisticRegression

In [37]:
# fit logistic regression model

lr = LogisticRegression(penalty='l2')

In [38]:
# shape check

train_data_features.shape, y_train.shape

((1296, 5000), (1296,))

In [39]:
lr.fit(train_data_features, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
lr.score(train_data_features, y_train)

0.9953703703703703

In [41]:
lr.score(test_data_features, y_test)

0.9583333333333334

### Feature comparison

Creates a dataframe that matches features to coefficients

In [42]:
coef_list = lr.coef_.tolist()

In [43]:
coef_list = coef_list[0]

In [44]:
coef_df = pd.DataFrame({'features': vectorizer.get_feature_names(),
                        'coefs': coef_list})

In [45]:
coef_df.sort_values(by = ['coefs'])

,features,coefs
2330,nba,-2.466467
1402,heat,-1.322496
1365,harden,-1.226266
4844,warriors,-1.195970
1780,lebron,-1.108881
1664,kat,-1.080393
2172,luka,-1.063615
2444,okc,-0.942514
4538,suns,-0.888450
2943,points,-0.851006


### Let's throw out these unfair words and rerun

In [46]:
stopwords = set(stopwords.words('english'))

extra_stopwords = ['nba', 'basketball', 'football', 'nfl']

stopwords.update(extra_stopwords)

In [47]:
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = stopwords,
                             max_features = 5000,
                             ngram_range = (1, 3))

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

((1296, 5000), (432, 5000))

In [48]:
lr.fit(train_data_features, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
lr.score(train_data_features, y_train)

0.9915123456790124

In [50]:
lr.score(test_data_features, y_test)

0.9421296296296297

In [51]:
coef_list = lr.coef_.tolist()
coef_list = coef_list[0]

coef_df = pd.DataFrame({'features' : vectorizer.get_feature_names(),
                       'coefs' : coef_list})

coef_df.sort_values(by = ['coefs'])

,features,coefs
1471,heat,-1.390041
4828,warriors,-1.225633
1436,harden,-1.177751
1866,lebron,-1.105873
2081,luka,-1.101096
1731,kat,-1.072424
2697,okc,-0.950810
2920,points,-0.901830
1759,kings,-0.874735
4676,tonight,-0.861127


### Decision Tree

In [52]:
from sklearn.tree import DecisionTreeClassifier

In [53]:
tree = DecisionTreeClassifier()

In [54]:
tree.fit(train_data_features, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [55]:
tree.score(train_data_features, y_train)

0.9992283950617284

In [56]:
tree.score(test_data_features, y_test)

0.8194444444444444

### Random Forest

In [57]:
from sklearn.ensemble import RandomForestClassifier

In [62]:
forest = RandomForestClassifier(n_estimators = 1000)

In [63]:
forest.fit(train_data_features, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [64]:
forest.score(train_data_features, y_train)

0.9992283950617284

In [65]:
forest.score(test_data_features, y_test)

0.8819444444444444

###  Matrix on Logistic Regression

In [66]:
from sklearn.metrics import confusion_matrix

In [67]:
y_pred = lr.predict(test_data_features)

In [68]:
cm = confusion_matrix(y_test, y_pred)

In [69]:
cm_df = pd.DataFrame(cm,
                    columns=['predict_neg', 'predict_pos'],
                    index = ['actual_neg', 'actual_pos'])

In [70]:
cm_df

,predict_neg,predict_pos
actual_neg,188,9
actual_pos,16,219


## Checking where our model failed

In [71]:
comparison_df = pd.DataFrame({'y_actual' : y_test,
             'y_predicted' : y_pred})

In [72]:
mismatch_df = comparison_df[comparison_df['y_actual'] != comparison_df['y_predicted']]

In [73]:
mismatch2_df = pd.concat([mismatch_df, X_test], axis = 1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [74]:
# All incorrect predictions with titles

mismatches = mismatch2_df.dropna()

In [75]:
mismatches

,y_actual,y_predicted,title
t3_egvgl9,nba,nfl,charania the nuggets has been open to discussing deals around young talents malik beasley and juancho hernangomez but have asked for a high value ahead of the february trade deadline according to sources both turned down extensions this past summer
t3_ehboan,nfl,nba,highlight tavon wilson called for unnecessary roughness
t3_ehc6su,nba,nfl,aside from the 2015 16 season what is the best nba season in nba history
t3_ehgzu1,nfl,nba,siciliano so game 256 comes down to one play
t3_ehiiue,nfl,nba,nfl draft order top 20 set
t3_ehlj3x,nfl,nba,which playoff game s will romo cast
t3_ehm1xy,nfl,nba,fans of the 8 teams playing this weekend how does everyone feel about their first round matchup
t3_ehnm3w,nfl,nba,jackson xavien howard arrest details
t3_ehnwvg,nfl,nba,person tre boston on cam newton the rest of those boys are playing to 40 but they got o lines why does cam get to have just average lines get beat up amp the rest of these guys have healthy o lines and get the same treatment as aw man these guys can play til they re 40 with ease
t3_ehpy1s,nba,nfl,stupid question why did the lakers sign cousins to a one year contract


### Let's try TF-IDF

Term Frequency / Inverse Document Frequency

TF(w) = (Number of times term w appears in a document) / (Total number of terms in the document)

IDF(w) = log_e(Total number of documents / Number of documents with term w in it)

In [83]:
tfidf_vec = TfidfVectorizer(analyzer="word",
                            tokenizer=None,
                            preprocessor=None,
                            stop_words=['nba', 'nfl', 'football', 'basketball'],
                            max_features=5000,
                            ngram_range=(1, 3))

In [84]:
train_data_features = tfidf_vec.fit_transform(clean_train_data)

test_data_features = tfidf_vec.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

((1296, 5000), (432, 5000))

In [85]:
lr.fit(train_data_features, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [86]:
lr.score(train_data_features, y_train)

0.9699074074074074

In [87]:
lr.score(test_data_features, y_test)

0.8888888888888888

### Let's try on some other subreddits

In [88]:
train = pd.concat([politics_test, conservative_test])

In [89]:
X = train[['title']]
y = train['subreddit']

In [90]:
# politics_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/politics.json')
# conservative_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/conservative.json')

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [92]:
politics_test = politics_test.drop(columns='selftext')
conservative_test = conservative_test.drop(columns='selftext')

train = pd.concat([politics_test, conservative_test])
tokenizer = RegexpTokenizer(r'\w+')

train['title'] = train['title'].map(lambda x: tokenizer.tokenize(x.lower()))
train['title'] = train['title'].map(lambda x: ' '.join(x))

In [93]:
# create our training data list - this is a list of strings, with each string being a post title

clean_train_data = []

for traindata in X_train['title']:
    clean_train_data.append(traindata)
    
    
# create test data list

clean_test_data = []

for testdata in X_test['title']:
    clean_test_data.append(testdata)

In [101]:
vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words='english',
                             max_features=5000,
                             ngram_range=(2, 3))

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

vocab = vectorizer.get_feature_names()

### Modeling

In [102]:
lr = LogisticRegression(penalty = 'l2')

In [103]:
train_data_features.shape, y_train.shape

((1343, 5000), (1343,))

In [104]:
lr.fit(train_data_features, y_train)

lr.score(train_data_features, y_train)

0.9538346984363366

In [105]:
lr.score(test_data_features, y_test)

0.6629464285714286

In [106]:
coef_list = lr.coef_.tolist()

coef_list = coef_list[0]

In [107]:
coef_df = pd.DataFrame({'features' : vectorizer.get_feature_names(),
                       'coefs' : coef_list})

coef_df.sort_values(by = ['coefs'])

,features,coefs
1077,merry christmas,-1.191564
417,hunter biden,-0.930663
4971,working class,-0.841837
409,hillary clinton,-0.772845
539,lisa murkowski,-0.744156
170,christmas message,-0.681478
303,fake news,-0.667685
74,articles impeachment,-0.665513
71,anti trump,-0.661449
34,advanced society,-0.650944


# Understanding word vectors

... for, like, actual poets. By [Allison Parrish](http://www.decontextualize.com/)


## Why word vectors?

Poetry is, at its core, the art of identifying and manipulating linguistic similarity. I have discovered a truly marvelous proof of this, which this notebook is too narrow to contain. (By which I mean: I will elaborate on this some other time)

## Animal similarity and simple linear algebra

We'll begin by considering a small subset of English: words for animals. Our task is to be able to write computer programs to find similarities among these words and the creatures they designate. To do this, we might start by making a spreadsheet of some animals and their characteristics. For example:

![Animal spreadsheet](http://static.decontextualize.com/snaps/animal-spreadsheet.png)

This spreadsheet associates a handful of animals with two numbers: their cuteness and their size, both in a range from zero to one hundred. (The values themselves are simply based on my own judgment. Your taste in cuteness and evaluation of size may differ significantly from mine. As with all data, these data are simply a mirror reflection of the person who collected them.)

These values give us everything we need to make determinations about which animals are similar (at least, similar in the properties that we've included in the data). Try to answer the following question: Which animal is most similar to a capybara? You could go through the values one by one and do the math to make that evaluation, but visualizing the data as points in 2-dimensional space makes finding the answer very intuitive:

![Animal space](http://static.decontextualize.com/snaps/animal-space.png)

The plot shows us that the closest animal to the capybara is the panda bear (again, in terms of its subjective size and cuteness). One way of calculating how "far apart" two points are is to find their *Euclidean distance*. (This is simply the length of the line that connects the two points.) For points in two dimensions, Euclidean distance can be calculated with the following Python function:

In [108]:
import numpy as np
def distance2d(x1, y1, x2, y2):
    return np.linalg.norm(np.array([x1, y1])-np.array([x2, y2]))

So, the distance between "capybara" (70, 30) and "panda" (74, 40):

In [109]:
distance2d(70, 30, 75, 40) # panda and capybara

11.180339887498949

... is less than the distance between "tarantula" and "elephant":

In [110]:
distance2d(8, 3, 65, 90) # tarantula and elephant

104.0096149401583

Modeling animals in this way has a few other interesting properties. For example, you can pick an arbitrary point in "animal space" and then find the animal closest to that point. If you imagine an animal of size 25 and cuteness 30, you can easily look at the space to find the animal that most closely fits that description: the chicken.

Reasoning visually, you can also answer questions like: what's halfway between a chicken and an elephant? Simply draw a line from "elephant" to "chicken," mark off the midpoint and find the closest animal. (According to our chart, halfway between an elephant and a chicken is a horse.)

You can also ask: what's the *difference* between a hamster and a tarantula? According to our plot, it's about seventy five units of cute (and a few units of size).

The relationship of "difference" is an interesting one, because it allows us to reason about *analogous* relationships. In the chart below, I've drawn an arrow from "tarantula" to "hamster" (in blue):

![Animal analogy](http://static.decontextualize.com/snaps/animal-space-analogy.png)

You can understand this arrow as being the *relationship* between a tarantula and a hamster, in terms of their size and cuteness (i.e., hamsters and tarantulas are about the same size, but hamsters are much cuter). In the same diagram, I've also transposed this same arrow (this time in red) so that its origin point is "chicken." The arrow ends closest to "kitten." What we've discovered is that the animal that is about the same size as a chicken but much cuter is... a kitten. To put it in terms of an analogy:

    Tarantulas are to hamsters as chickens are to kittens.
    
A sequence of numbers used to identify a point is called a *vector*, and the kind of math we've been doing so far is called *linear algebra.* (Linear algebra is surprisingly useful across many domains: It's the same kind of math you might do to, e.g., simulate the velocity and acceleration of a sprite in a video game.)

A set of vectors that are all part of the same data set is often called a *vector space*. The vector space of animals in this section has two *dimensions*, by which I mean that each vector in the space has two numbers associated with it (i.e., two columns in the spreadsheet). The fact that this space has two dimensions just happens to make it easy to *visualize* the space by drawing a 2D plot. But most vector spaces you'll work with will have more than two dimensions—sometimes many hundreds. In those cases, it's more difficult to visualize the "space," but the math works pretty much the same.

## Language with vectors: colors

So far, so good. We have a system in place—albeit highly subjective—for talking about animals and the words used to name them. I want to talk about another vector space that has to do with language: the vector space of colors.

Colors are often represented in computers as vectors with three dimensions: red, green, and blue. Just as with the animals in the previous section, we can use these vectors to answer questions like: which colors are similar? What's the most likely color name for an arbitrarily chosen set of values for red, green and blue? Given the names of two colors, what's the name of those colors' "average"?

We'll be working with this [color data](https://github.com/dariusk/corpora/blob/master/data/colors/xkcd.json) from the [xkcd color survey](https://blog.xkcd.com/2010/05/03/color-survey-results/). The data relates a color name to the RGB value associated with that color. [Here's a page that shows what the colors look like](https://xkcd.com/color/rgb/). Download the color data and put it in the same directory as this notebook.

A few notes before we proceed:

* The linear algebra functions implemented below (`addv`, `meanv`, etc.) are slow, potentially inaccurate, and shouldn't be used for "real" code—I wrote them so beginner programmers can understand how these kinds of functions work behind the scenes. Use [numpy](http://www.numpy.org/) for fast and accurate math in Python.
* If you're interested in perceptually accurate color math in Python, consider using the [colormath library](http://python-colormath.readthedocs.io/en/latest/).

Now, import the `json` library and load the color data:

In [121]:
import requests

In [122]:
resp = requests.get('https://raw.githubusercontent.com/dariusk/corpora/master/data/colors/xkcd.json')
color_data = resp.json()

The following function converts colors from hex format (`#1a2b3c`) to a tuple of integers:

In [123]:
def hex_to_int(s):
    return int(s[1:3], 16), int(s[3:5], 16), int(s[5:7], 16)

And the following cell creates a dictionary and populates it with mappings from color names to RGB vectors for each color in the data:

In [124]:
colors = dict()
for item in color_data['colors']:
    colors[item["color"]] = hex_to_int(item["hex"])

Testing it out:

In [125]:
colors['olive']

(110, 117, 14)

In [126]:
colors['red']

(229, 0, 0)

In [127]:
colors['black']

(0, 0, 0)

In [128]:
colors.keys()

dict_keys(['cloudy blue', 'dark pastel green', 'dust', 'electric lime', 'fresh green', 'light eggplant', 'nasty green', 'really light blue', 'tea', 'warm purple', 'yellowish tan', 'cement', 'dark grass green', 'dusty teal', 'grey teal', 'macaroni and cheese', 'pinkish tan', 'spruce', 'strong blue', 'toxic green', 'windows blue', 'blue blue', 'blue with a hint of purple', 'booger', 'bright sea green', 'dark green blue', 'deep turquoise', 'green teal', 'strong pink', 'bland', 'deep aqua', 'lavender pink', 'light moss green', 'light seafoam green', 'olive yellow', 'pig pink', 'deep lilac', 'desert', 'dusty lavender', 'purpley grey', 'purply', 'candy pink', 'light pastel green', 'boring green', 'kiwi green', 'light grey green', 'orange pink', 'tea green', 'very light brown', 'egg shell', 'eggplant purple', 'powder pink', 'reddish grey', 'baby shit brown', 'liliac', 'stormy blue', 'ugly brown', 'custard', 'darkish pink', 'deep brown', 'greenish beige', 'manilla', 'off blue', 'battleship gre

### Vector math

Before we keep going, we'll need some functions for performing basic vector arithmetic. These functions will work with vectors in spaces of any number of dimensions.

The first function returns the Euclidean distance between two points:

In [129]:
import math
def distance(coord1, coord2):
    return np.linalg.norm(np.array(coord1)-np.array(coord2))
distance([10, 1], [5, 2])

5.0990195135927845

The `subtractv` function subtracts one vector from another:

In [130]:
def subtractv(coord1, coord2):
    return list(np.array(coord1) - np.array(coord2))
subtractv([10, 1], [5, 2])

[5, -1]

The `addv` vector adds two vectors together:

In [131]:
def addv(coord1, coord2):
    return list(np.array(coord1) + np.array(coord2))
addv([10, 1], [5, 2])

[15, 3]

And the `meanv` function takes a list of vectors and finds their mean or average:

In [132]:
def meanv(coords):
    return list(np.mean(coords, axis=0))
meanv([[0, 1], [2, 2], [4, 3]])

[2.0, 2.0]

Just as a test, the following cell shows that the distance from "red" to "green" is greater than the distance from "red" to "pink":

In [133]:
distance(colors['red'], colors['green']) > distance(colors['red'], colors['pink'])

True

### Finding the closest item

Just as we wanted to find the animal that most closely matched an arbitrary point in cuteness/size space, we'll want to find the closest color name to an arbitrary point in RGB space. The easiest way to find the closest item to an arbitrary vector is simply to find the distance between the target vector and each item in the space, in turn, then sort the list from closest to farthest. The `closest()` function below does just that. By default, it returns a list of the ten closest items to the given vector.

> Note: Calculating "closest neighbors" like this is fine for the examples in this notebook, but unmanageably slow for vector spaces of any appreciable size. As your vector space grows, you'll want to move to a faster solution, like SciPy's [kdtree](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.KDTree.html) or [Annoy](https://pypi.python.org/pypi/annoy).

In [134]:
def closest(space, coord, n=10):
    return sorted(space.keys(), key=lambda x: distance(coord, space[x]))[:n]

Testing it out, we can find the ten colors closest to "red":

In [151]:
closest(colors, colors['white'])

['white',
 'pale grey',
 'very light pink',
 'off white',
 'ice blue',
 'very pale blue',
 'ice',
 'very light blue',
 'really light blue',
 'eggshell']

... or the ten colors closest to (150, 60, 150):

In [152]:
closest(colors, [255, 60, 150])

['barbie pink',
 'warm pink',
 'medium pink',
 'rosy pink',
 'darkish pink',
 'bubble gum pink',
 'strong pink',
 'electric pink',
 'bubblegum',
 'hot pink']

### Color magic

The magical part of representing words as vectors is that the vector operations we defined earlier appear to operate on language the same way they operate on numbers. For example, if we find the word closest to the vector resulting from subtracting "red" from "purple," we get a series of "blue" colors:

In [153]:
closest(colors, subtractv(colors['green'], colors['purple']))

['true green',
 'racing green',
 'bottle green',
 'deep green',
 'darkgreen',
 'forest',
 'emerald green',
 'dark green',
 'vibrant green',
 'british racing green']

This matches our intuition about RGB colors, which is that purple is a combination of red and blue. Take away the red, and blue is all you have left.

You can do something similar with addition. What's blue plus green?

In [154]:
closest(colors, addv(colors['blue'], colors['green']))

['bright turquoise',
 'bright light blue',
 'bright aqua',
 'cyan',
 'neon blue',
 'aqua blue',
 'bright cyan',
 'bright sky blue',
 'aqua',
 'bright teal']

That's right, it's something like turquoise or cyan! What if we find the average of black and white? Predictably, we get gray:

In [155]:
# the average of black and white: medium grey
closest(colors, meanv([colors['black'], colors['white']]))

['medium grey',
 'purple grey',
 'steel grey',
 'battleship grey',
 'grey purple',
 'purplish grey',
 'greyish purple',
 'steel',
 'warm grey',
 'green grey']

Just as with the tarantula/hamster example from the previous section, we can use color vectors to reason about relationships between colors. In the cell below, finding the difference between "pink" and "red" then adding it to "blue" seems to give us a list of colors that are to blue what pink is to red (i.e., a slightly lighter, less saturated shade):

In [156]:
# an analogy: pink is to red as X is to blue
pink_to_red = subtractv(colors['orange'], colors['red'])
closest(colors, addv(pink_to_red, colors['blue']))

['azure',
 'turquoise blue',
 'bright sky blue',
 'aqua blue',
 'neon blue',
 'water blue',
 'topaz',
 'cerulean',
 'aqua',
 'dark sky blue']

Another example of color analogies: Navy is to blue as true green/dark grass green is to green:

In [157]:
# another example: 
navy_to_blue = subtractv(colors['navy'], colors['blue'])
closest(colors, addv(navy_to_blue, colors['green']))

['true green',
 'dark grass green',
 'grassy green',
 'racing green',
 'forest',
 'bottle green',
 'dark olive green',
 'darkgreen',
 'forrest green',
 'grass green']

The examples above are fairly simple from a mathematical perspective but nevertheless *feel* magical: they're demonstrating that it's possible to use math to reason about how people use language.

### Interlude: A Love Poem That Loses Its Way

In [158]:
import random
red = colors['red']
blue = colors['blue']
for i in range(14):
    rednames = closest(colors, red)
    bluenames = closest(colors, blue)
    print(f"Roses are {rednames[0]}, violets are {bluenames[0]}")
    red = colors[random.choice(rednames[1:])]
    blue = colors[random.choice(bluenames[1:])]

Roses are red, violets are blue
Roses are tomato red, violets are electric blue
Roses are red orange, violets are azul
Roses are orangish red, violets are bright blue
Roses are tomato red, violets are deep sky blue
Roses are deep orange, violets are bright blue
Roses are dark orange, violets are deep sky blue
Roses are rust orange, violets are cerulean blue
Roses are burnt orange, violets are bright blue
Roses are brick orange, violets are electric blue
Roses are burnt siena, violets are vibrant blue
Roses are orangey brown, violets are bright blue
Roses are caramel, violets are electric blue
Roses are orange brown, violets are vivid blue


### Doing bad digital humanities with color vectors

With the tools above in hand, we can start using our vectorized knowledge of language toward academic ends. In the following example, I'm going to calculate the average color of Bram Stoker's *Dracula*.

First, we'll load [spaCy](https://spacy.io/):

In [143]:
import spacy
nlp = spacy.load('en_core_web_sm')

To calculate the average color, we'll follow these steps:

1. Parse the text into words
2. Check every word to see if it names a color in our vector space. If it does, add it to a list of vectors.
3. Find the average of that list of vectors.
4. Find the color(s) closest to that average vector.

The following cell performs steps 1-3:

In [162]:
resp = requests.get('http://www.gutenberg.org/cache/epub/345/pg345.txt')
dracula = nlp(resp.text)
# use word.lower_ to normalize case
drac_colors = [colors[word.lower_] for word in dracula if word.lower_ in colors]
avg_color = meanv(drac_colors)
print(avg_color)

[147.44839067702551, 113.65371809100999, 100.13540510543841]


Now, we'll pass the averaged color vector to the `closest()` function, yielding... well, it's just a brown mush, which is kinda what you'd expect from adding a bunch of colors together willy-nilly.

In [163]:
closest(colors, avg_color)

['reddish grey',
 'brownish grey',
 'brownish',
 'brown grey',
 'mocha',
 'grey brown',
 'puce',
 'dull brown',
 'pinkish brown',
 'dark taupe']

On the other hand, here's what we get when we average the colors of Charlotte Perkins Gilman's classic *The Yellow Wallpaper*. The result definitely reflects the content of the story, so maybe we're on to something here.

In [161]:
resp = requests.get('http://www.gutenberg.org/cache/epub/1952/pg1952.txt')
yellow = nlp(resp.text)
wallpaper_colors = [colors[word.lower_] for word in yellow if word.lower_ in colors]
avg_color = meanv(wallpaper_colors)
closest(colors, avg_color)

['sickly yellow',
 'piss yellow',
 'puke yellow',
 'vomit yellow',
 'dirty yellow',
 'mustard yellow',
 'dark yellow',
 'olive yellow',
 'macaroni and cheese',
 'pea']

Exercise for the reader: Use the vector arithmetic functions to rewrite a text, making it...

* more blue (i.e., add `colors['blue']` to each occurrence of a color word); or
* more light (i.e., add `colors['white']` to each occurrence of a color word); or
* darker (i.e., attenuate each color. You might need to write a vector multiplication function to do this one right.)

## Distributional semantics

In the previous section, the examples are interesting because of a simple fact: colors that we think of as similar are "closer" to each other in RGB vector space. In our color vector space, or in our animal cuteness/size space, you can think of the words identified by vectors close to each other as being *synonyms*, in a sense: they sort of "mean" the same thing. They're also, for many purposes, *functionally identical*. Think of this in terms of writing, say, a search engine. If someone searches for "mauve trousers," then it's probably also okay to show them results for, say,

In [147]:
for cname in closest(colors, colors['mauve']):
    print(cname, "trousers")

mauve trousers
dusty rose trousers
dusky rose trousers
brownish pink trousers
old pink trousers
reddish grey trousers
dirty pink trousers
old rose trousers
light plum trousers
ugly pink trousers


That's all well and good for color words, which intuitively seem to exist in a multidimensional continuum of perception, and for our animal space, where we've written out the vectors ahead of time. But what about... arbitrary words? Is it possible to create a vector space for all English words that has this same "closer in space is closer in meaning" property?

To answer that, we have to back up a bit and ask the question: what does *meaning* mean? No one really knows, but one theory popular among computational linguists, computer scientists and other people who make search engines is the [Distributional Hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics), which states that:

    Linguistic items with similar distributions have similar meanings.
    
What's meant by "similar distributions" is *similar contexts*. Take for example the following sentences:

    It was really cold yesterday.
    It will be really warm today, though.
    It'll be really hot tomorrow!
    Will it be really cool Tuesday?
    
According to the Distributional Hypothesis, the words `cold`, `warm`, `hot` and `cool` must be related in some way (i.e., be close in meaning) because they occur in a similar context, i.e., between the word "really" and a word indicating a particular day. (Likewise, the words `yesterday`, `today`, `tomorrow` and `Tuesday` must be related, since they occur in the context of a word indicating a temperature.)

In other words, according to the Distributional Hypothesis, a word's meaning is just a big list of all the contexts it occurs in. Two words are closer in meaning if they share contexts.

## Word vectors by counting contexts

So how do we turn this insight from the Distributional Hypothesis into a system for creating general-purpose vectors that capture the meaning of words? Maybe you can see where I'm going with this. What if we made a *really big* spreadsheet that had one column for every context for every word in a given source text. Let's use a small source text to begin with, such as this excerpt from Dickens:

    It was the best of times, it was the worst of times.

Such a spreadsheet might look something like this:

![dickens contexts](http://static.decontextualize.com/snaps/best-of-times.png)

The spreadsheet has one column for every possible context, and one row for every word. The values in each cell correspond with how many times the word occurs in the given context. The numbers in the columns constitute that word's vector, i.e., the vector for the word `of` is

    [0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
    
Because there are ten possible contexts, this is a ten dimensional space! It might be strange to think of it, but you can do vector arithmetic on vectors with ten dimensions just as easily as you can on vectors with two or three dimensions, and you could use the same distance formula that we defined earlier to get useful information about which vectors in this space are similar to each other. In particular, the vectors for `best` and `worst` are actually the same (a distance of zero), since they occur only in the same context (`the ___ of`):

    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
    
Of course, the conventional way of thinking about "best" and "worst" is that they're *antonyms*, not *synonyms*. But they're also clearly two words of the same kind, with related meanings (through opposition), a fact that is captured by this distributional model.

### Contexts and dimensionality

Of course, in a corpus of any reasonable size, there will be many thousands if not many millions of possible contexts. It's difficult enough working with a vector space of ten dimensions, let alone a vector space of a million dimensions! It turns out, though, that many of the dimensions end up being superfluous and can either be eliminated or combined with other dimensions without significantly affecting the predictive power of the resulting vectors. The process of getting rid of superfluous dimensions in a vector space is called [dimensionality reduction](https://en.wikipedia.org/wiki/Dimensionality_reduction), and most implementations of count-based word vectors make use of dimensionality reduction so that the resulting vector space has a reasonable number of dimensions (say, 100—300, depending on the corpus and application).

The question of how to identify a "context" is itself very difficult to answer. In the toy example above, we've said that a "context" is just the word that precedes and the word that follows. Depending on your implementation of this procedure, though, you might want a context with a bigger "window" (e.g., two words before and after), or a non-contiguous window (skip a word before and after the given word). You might exclude certain "function" words like "the" and "of" when determining a word's context, or you might [lemmatize](https://en.wikipedia.org/wiki/Lemmatisation) the words before you begin your analysis, so two occurrences with different "forms" of the same word count as the same context. These are all questions open to research and debate, and different implementations of procedures for creating count-based word vectors make different decisions on this issue.

### GloVe vectors

But you don't have to create your own word vectors from scratch! Many researchers have made downloadable databases of pre-trained vectors. One such project is Stanford's [Global Vectors for Word Representation (GloVe)](https://nlp.stanford.edu/projects/glove/). These 300-dimensional vectors are included with spaCy, and they're the vectors we'll be using for the rest of this tutorial.

## Word vectors in spaCy

Okay, let's have some fun with real word vectors. We're going to use the GloVe vectors that come with spaCy to creatively analyze and manipulate the text of Bram Stoker's *Dracula*. First, make sure you've got `spacy` imported:

In [164]:
dracula[560]

hour

In [165]:
_.vector

array([-3.3890932 ,  3.8036323 , -2.6519728 ,  1.6794462 , -0.03245598,
        0.34949422,  1.4263201 ,  0.7514826 ,  1.5714422 ,  6.836588  ,
       -0.87871516, -2.5962636 ,  4.4980946 , -3.2636712 ,  0.59583217,
        1.7193251 , -1.7698543 , -2.6230168 ,  0.32886952, -1.9750781 ,
        3.1977305 , -2.3372014 , -2.340062  , -2.59442   , -1.707979  ,
        1.4709201 ,  0.46112838, -4.2141914 ,  3.8767452 ,  1.9445885 ,
        2.5187936 ,  2.709597  ,  4.9126153 , -2.4080238 ,  1.2024103 ,
       -1.7131364 ,  2.7877707 , -4.4838257 , -1.9192042 , -2.094125  ,
        3.3191376 ,  2.954278  , -1.6426673 , -2.4524295 , -0.97719395,
        1.2865635 , -0.9342208 , -0.07476962, -1.0927347 ,  0.39212835,
        2.6349268 , -3.4252431 ,  0.7364484 , -0.04758237, -0.8591391 ,
        0.5237043 ,  3.3768516 , -0.6579486 , -0.34316725, -0.7823252 ,
        2.7190056 , -2.2406356 , -1.2399366 , -0.10923919,  0.38356325,
       -0.07189876,  4.31141   ,  2.5327733 ,  0.69408464, -0.58

In [167]:
# previously we've used the _sm model, which doesn't include all vectors.
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.0/en_core_web_lg-2.2.0.tar.gz
import en_core_web_lg
nlp = en_core_web_lg.load()

  Stored in directory: C:\Users\BudBe\AppData\Local\pip\Cache\wheels\9f\3c\d6\3ade7ed8195030f4d7f299cf73d856a84d7b3effd5890133fb
Successfully built en-core-web-lg


In [168]:
resp = requests.get('http://www.gutenberg.org/cache/epub/345/pg345.txt')
dracula = nlp(resp.text)


And the cell below creates a list of unique words (or tokens) in the text, as a list of strings.

In [169]:
# all of the words in the text file
tokens = list(set([w.text for w in dracula if w.is_alpha]))

You can see the vector of any word in spaCy's vocabulary using the `vocab` attribute, like so:

In [170]:
nlp.vocab['alligator'].vector

array([-4.9304e-01,  1.2459e-01,  2.6142e-01,  3.8393e-02,  1.4018e-01,
        5.7285e-01, -5.0449e-01,  1.5094e-01,  7.3356e-02,  2.3308e-01,
       -1.6048e-01, -5.1184e-01, -1.4028e-01, -3.0110e-01, -4.3446e-01,
       -2.8257e-02,  1.9100e-03,  6.5157e-01,  1.4855e-01, -3.7255e-01,
       -4.6619e-01, -1.7223e-01, -5.1794e-01, -2.5453e-01, -2.6785e-01,
       -6.7776e-02, -3.0085e-01, -3.3212e-01, -1.5862e-01, -1.2336e-01,
       -3.8935e-01, -3.5551e-01, -3.6182e-01,  2.3197e-02, -1.7486e-01,
        2.2345e-01,  6.7557e-01,  9.6939e-03, -9.5640e-02,  1.2073e-01,
       -1.2016e-01, -1.6161e-01,  1.0661e-01, -4.5233e-01,  4.5991e-01,
        1.0367e-02, -1.0842e-01, -3.6163e-01,  2.5105e-01,  9.5780e-03,
       -6.1363e-01,  2.7494e-01,  2.1783e-01, -1.0663e-01, -9.5146e-03,
        2.7570e-01,  2.5888e-02, -5.6749e-02,  6.7667e-03,  5.3735e-01,
       -1.2224e-01,  8.8448e-02,  4.6094e-01,  4.6881e-02, -6.6557e-01,
       -3.9493e-01,  2.7362e-01,  1.2302e-01,  2.8594e-01, -2.52

For the sake of convenience, the following function gets the vector of a given string from spaCy's vocabulary:

In [171]:
def vec(s):
    return nlp.vocab[s].vector

### Cosine similarity and finding closest neighbors

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

The following cell shows that the cosine similarity between `dog` and `puppy` is larger than the similarity between `trousers` and `octopus`, thereby demonstrating that the vectors are working how we expect them to:

In [ ]:
(cosine_similarity(vec('dog').reshape(1, -1), vec('puppy').reshape(1, -1))[0][0] > 
    cosine_similarity(vec('trousers').reshape(1, -1), vec('octopus').reshape(1, -1))[0][0])

The following cell defines a function that iterates through a list of tokens and returns the token whose vector is most similar to a given vector.

In [ ]:
def spacy_closest(token_list, vec_to_check, n=10):
    return sorted(token_list,
                  key=lambda x: cosine_similarity(vec_to_check.reshape(1, -1), 
                                                  vec(x).reshape(1, -1))[0][0],
                  reverse=True)[:n]

Using this function, we can get a list of synonyms, or words closest in meaning (or distribution, depending on how you look at it), to any arbitrary word in spaCy's vocabulary. In the following example, we're finding the words in *Dracula* closest to "basketball":

In [ ]:
# what's the closest equivalent of basketball?
spacy_closest(tokens, vec("basketball"))

### Fun with spaCy, Dracula, and vector arithmetic

Now we can start doing vector arithmetic and finding the closest words to the resulting vectors. For example, what word is closest to the halfway point between day and night?

In [ ]:
# halfway between day and night
spacy_closest(tokens, np.array(meanv([vec("day"), vec("night")])))

Variations of `night` and `day` are still closest, but after that we get words like `evening` and `morning`, which are indeed halfway between day and night!

Here are the closest words in _Dracula_ to "wine":

In [ ]:
spacy_closest(tokens, vec("wine"))

If you subtract "alcohol" from "wine" and find the closest words to the resulting vector, you're left with simply a lovely dinner:

In [ ]:
spacy_closest(tokens, np.array(subtractv(vec("wine"), vec("alcohol"))))

The closest words to "water":

In [ ]:
spacy_closest(tokens, vec("water"))

But if you add "frozen" to "water," you get "ice":

In [ ]:
spacy_closest(tokens, np.array(addv(vec("water"), vec("frozen"))))

You can even do analogies! For example, the words most similar to "grass":

In [ ]:
spacy_closest(tokens, vec("grass"))

If you take the difference of "blue" and "sky" and add it to grass, you get the analogous word ("green"):

In [ ]:
# analogy: blue is to sky as X is to grass
blue_to_sky = subtractv(vec("blue"), vec("sky"))
spacy_closest(tokens, np.array(addv(blue_to_sky, vec("grass"))))

## Further resources

* [Word2vec](https://en.wikipedia.org/wiki/Word2vec) is another procedure for producing word vectors which uses a predictive approach rather than a context-counting approach. [This paper](http://clic.cimec.unitn.it/marco/publications/acl2014/baroni-etal-countpredict-acl2014.pdf) compares and contrasts the two approaches. (Spoiler: it's kind of a wash.)
* If you want to train your own word vectors on a particular corpus, the popular Python library [gensim](https://radimrehurek.com/gensim/) has an implementation of Word2Vec that is relatively easy to use. [There's a good tutorial here.](https://rare-technologies.com/word2vec-tutorial/)
* When you're working with vector spaces with high dimensionality and millions of vectors, iterating through your entire space calculating cosine similarities can be a drag. I use [Annoy](https://pypi.python.org/pypi/annoy) to make these calculations faster, and you should consider using it too.